<summary>Table of Contents</summary>

- [1. Connecting to CUDA](#1-connecting-to-cuda)
- [2. Informer](#2-informer)
- [3. PatchTST 336](#3-patchtst-336)
- [4. PatchTST 512](#4-patchtst-512)


Script with Informer and PatchTST (default parameters). PatchTST with seq_len = 336 and 512.

In [1]:
# Import libraries
import torch
import pandas as pd
import subprocess
import os
import time
import shutil

from utils.helper import extract_metrics_from_output, running_time, convert_results_into_df

# 1. Connecting to CUDA

Run on GPU, because running it on CPU will cost a lot of time.


I do not recommend to run it in Google Colab, because it interrupts training process.

If you are not going to use remote servers with multiple GPUs, skip this part.

In [2]:
# For CUDA making it available this works:
# pip3 install torch torchvision torchaudio

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [3]:
# Check the number of available GPUs
num_gpus = torch.cuda.device_count()
print("Number of available GPUs:", num_gpus)

Number of available GPUs: 3


In [4]:
torch.cuda.get_device_name(1)

'Tesla V100-PCIE-32GB'

In [5]:
# Index of the GPU you want to use (e.g., 0, 1, 2, etc.)
# Choose that one that is not used by other processes
cuda_device = "1"

# 2. Informer 168

In [7]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/informer/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [8]:
# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['IT']
num_cols = [3]
seq_len = 168
model = "Informer"
itr = 2

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_168.log"

# Parameters for tuning
lr = 0.0001
#n_heads = 16
e_layers = 2
d_layers = 1
loss = "MAE"

os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results = []

In [ ]:
DE 24
Scaled mse:0.025699591264128685, rmse:0.16031092405319214, mae:0.10348266363143921, rse:0.5661423206329346
Scaled mse:0.023365361616015434, rmse:0.15285731852054596, mae:0.10056539624929428, rse:0.5398197770118713

DE 96
Scaled mse:0.04860777407884598, rmse:0.22047170996665955, mae:0.14964795112609863, rse:0.7807348966598511
Scaled mse:0.05066155269742012, rmse:0.22508122026920319, mae:0.1532352715730667, rse:0.79705810546875

DE 168
Scaled mse:0.054843734949827194, rmse:0.23418739438056946, mae:0.15871629118919373, rse:0.8296554088592529
Scaled mse:0.05288044735789299, rmse:0.2299574911594391, mae:0.15663425624370575, rse:0.814670205116272

GB 24
Scaled mse:0.046120475977659225, rmse:0.2147567868232727, mae:0.1404818892478943, rse:0.7404764294624329
Scaled mse:0.039505913853645325, rmse:0.19876094162464142, mae:0.13264088332653046, rse:0.6853231191635132

GB 96
Scaled mse:0.05855736881494522, rmse:0.24198630452156067, mae:0.16477428376674652, rse:0.8368223309516907
Scaled mse:0.05540859326720238, rmse:0.235390305519104, mae:0.16324692964553833, rse:0.8140124678611755

GB 168
Scaled mse:0.06292843818664551, rmse:0.250855416059494, mae:0.17068493366241455, rse:0.8696832656860352
Scaled mse:0.06641010195016861, rmse:0.257701575756073, mae:0.177326962351799, rse:0.8934180736541748

ES 24
Scaled mse:0.027730144560337067, rmse:0.16652370989322662, mae:0.10189637541770935, rse:0.4892970025539398
Scaled mse:0.03074352815747261, rmse:0.17533832788467407, mae:0.10366739332675934, rse:0.5151970386505127

ES 96
Scaled mse:0.048280902206897736, rmse:0.21972915530204773, mae:0.13662198185920715, rse:0.6454980969429016
Scaled mse:0.05975188687443733, rmse:0.24444198608398438, mae:0.1502540409564972, rse:0.7180969715118408

ES 168
Scaled mse:0.07949484884738922, rmse:0.2819482982158661, mae:0.1783347725868225, rse:0.828172504901886
Scaled mse:0.0831800326704979, rmse:0.2884095013141632, mae:0.18242256343364716, rse:0.8471510410308838

FR 24
Scaled mse:0.012692660093307495, rmse:0.1126617044210434, mae:0.06498495489358902, rse:0.4347411096096039
Scaled mse:0.013476599007844925, rmse:0.11608875542879105, mae:0.06680411100387573, rse:0.44796544313430786

FR 96
Scaled mse:0.022180521860718727, rmse:0.14893126487731934, mae:0.09044930338859558, rse:0.5761057138442993
Scaled mse:0.02346043847501278, rmse:0.15316800773143768, mae:0.09117613732814789, rse:0.592494547367096

FR 168
Scaled mse:0.026063205674290657, rmse:0.16144102811813354, mae:0.10040990263223648, rse:0.6252502799034119
Scaled mse:0.026005728170275688, rmse:0.16126291453838348, mae:0.10041678696870804, rse:0.6245604753494263



In [ ]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --d_layers {d_layers} \
              --factor 5 \
              --enc_in {num_cols[i]} \
              --dec_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --dropout 0.1 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --overlapping_windows \
              --loss_fnc "{loss}" \
              --scaler_type minmax \
              --if_relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                informer_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_168_24_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=168, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=5, learning_rate=0.0

In [9]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --d_layers {d_layers} \
              --factor 5 \
              --enc_in {num_cols[i]} \
              --dec_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --dropout 0.1 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --overlapping_windows \
              --loss_fnc "{loss}" \
              --scaler_type minmax \
              --if_relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                informer_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: IT ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_168_24_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=168, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=5, learning_rate=0.0

In [10]:
informer_results

[{'Country': 'IT',
  'Pred_len': 24,
  'Iteration': 1,
  'MSE': 0.013071063905954361,
  'RMSE': 0.11432875692844391,
  'MAE': 0.06469322741031647},
 {'Country': 'IT',
  'Pred_len': 24,
  'Iteration': 2,
  'MSE': 0.011997658759355545,
  'RMSE': 0.10953382402658463,
  'MAE': 0.06481219828128815},
 {'Country': 'IT',
  'Pred_len': 96,
  'Iteration': 1,
  'MSE': 0.022102830931544304,
  'RMSE': 0.14867021143436432,
  'MAE': 0.09126447886228561},
 {'Country': 'IT',
  'Pred_len': 96,
  'Iteration': 2,
  'MSE': 0.022342106327414513,
  'RMSE': 0.14947275817394257,
  'MAE': 0.0909552201628685},
 {'Country': 'IT',
  'Pred_len': 168,
  'Iteration': 1,
  'MSE': 0.022543294355273247,
  'RMSE': 0.15014424920082092,
  'MAE': 0.09787493199110031},
 {'Country': 'IT',
  'Pred_len': 168,
  'Iteration': 2,
  'MSE': 0.02358420379459858,
  'RMSE': 0.1535715013742447,
  'MAE': 0.0978248119354248}]

In [ ]:
res= informer_results = []

# DE 24
informer_results.append({
    'Country': 'DE',
    'Pred_len': 24,
    'Iteration': 1,
    'MSE': 0.025699591264128685,
    'RMSE': 0.16031092405319214,
    'MAE': 0.10348266363143921,
})
informer_results.append({
    'Country': 'DE',
    'Pred_len': 24,
    'Iteration': 2,
    'MSE': 0.023365361616015434,
    'RMSE': 0.15285731852054596,
    'MAE': 0.10056539624929428,
})

# DE 96
informer_results.append({
    'Country': 'DE',
    'Pred_len': 96,
    'Iteration': 1,
    'MSE': 0.04860777407884598,
    'RMSE': 0.22047170996665955,
    'MAE': 0.14964795112609863,
})
informer_results.append({
    'Country': 'DE',
    'Pred_len': 96,
    'Iteration': 2,
    'MSE': 0.05066155269742012,
    'RMSE': 0.22508122026920319,
    'MAE': 0.1532352715730667,
})

# DE 168
informer_results.append({
    'Country': 'DE',
    'Pred_len': 168,
    'Iteration': 1,
    'MSE': 0.054843734949827194,
    'RMSE': 0.23418739438056946,
    'MAE': 0.15871629118919373,
})
informer_results.append({
    'Country': 'DE',
    'Pred_len': 168,
    'Iteration': 2,
    'MSE': 0.05288044735789299,
    'RMSE': 0.2299574911594391,
    'MAE': 0.15663425624370575,
})

# GB 24
informer_results.append({
    'Country': 'GB',
    'Pred_len': 24,
    'Iteration': 1,
    'MSE': 0.046120475977659225,
    'RMSE': 0.2147567868232727,
    'MAE': 0.1404818892478943,
})
informer_results.append({
    'Country': 'GB',
    'Pred_len': 24,
    'Iteration': 2,
    'MSE': 0.039505913853645325,
    'RMSE': 0.19876094162464142,
    'MAE': 0.13264088332653046,
})

# GB 96
informer_results.append({
    'Country': 'GB',
    'Pred_len': 96,
    'Iteration': 1,
    'MSE': 0.05855736881494522,
    'RMSE': 0.24198630452156067,
    'MAE': 0.16477428376674652,
})
informer_results.append({
    'Country': 'GB',
    'Pred_len': 96,
    'Iteration': 2,
    'MSE': 0.05540859326720238,
    'RMSE': 0.235390305519104,
    'MAE': 0.16324692964553833,
})

# GB 168
informer_results.append({
    'Country': 'GB',
    'Pred_len': 168,
    'Iteration': 1,
    'MSE': 0.06292843818664551,
    'RMSE': 0.250855416059494,
    'MAE': 0.17068493366241455,
})
informer_results.append({
    'Country': 'GB',
    'Pred_len': 168,
    'Iteration': 2,
    'MSE': 0.06641010195016861,
    'RMSE': 0.257701575756073,
    'MAE': 0.177326962351799,
})

# ES 24
informer_results.append({
    'Country': 'ES',
    'Pred_len': 24,
    'Iteration': 1,
    'MSE': 0.027730144560337067,
    'RMSE': 0.16652370989322662,
    'MAE': 0.10189637541770935,
})
informer_results.append({
    'Country': 'ES',
    'Pred_len': 24,
    'Iteration': 2,
    'MSE': 0.03074352815747261,
    'RMSE': 0.17533832788467407,
    'MAE': 0.10366739332675934,
})

# ES 96
informer_results.append({
    'Country': 'ES',
    'Pred_len': 96,
    'Iteration': 1,
    'MSE': 0.048280902206897736,
    'RMSE': 0.21972915530204773,
    'MAE': 0.13662198185920715,
})
informer_results.append({
    'Country': 'ES',
    'Pred_len': 96,
    'Iteration': 2,
    'MSE': 0.05975188687443733,
    'RMSE': 0.24444198608398438,
    'MAE': 0.1502540409564972,
})

# ES 168
informer_results.append({
    'Country': 'ES',
    'Pred_len': 168,
    'Iteration': 1,
    'MSE': 0.07949484884738922,
    'RMSE': 0.2819482982158661,
    'MAE': 0.1783347725868225,
})
informer_results.append({
    'Country': 'ES',
    'Pred_len': 168,
    'Iteration': 2,
    'MSE': 0.0831800326704979,
    'RMSE': 0.2884095013141632,
    'MAE': 0.18242256343364716,
})

# FR 24
informer_results.append({
    'Country': 'FR',
    'Pred_len': 24,
    'Iteration': 1,
    'MSE': 0.012692660093307495,
    'RMSE': 0.1126617044210434,
    'MAE': 0.06498495489358902,
})
informer_results.append({
    'Country': 'FR',
    'Pred_len': 24,
    'Iteration': 2,
    'MSE': 0.013476599007844925,
    'RMSE': 0.11608875542879105,
    'MAE': 0.06680411100387573,
})

# FR 96
informer_results.append({
    'Country': 'FR',
    'Pred_len': 96,
    'Iteration': 1,
    'MSE': 0.022180521860718727,
    'RMSE': 0.14893126487731934,
    'MAE': 0.09044930338859558,
})
informer_results.append({
    'Country': 'FR',
    'Pred_len': 96,
    'Iteration': 2,
    'MSE': 0.02346043847501278,
    'RMSE': 0.15316800773143768,
    'MAE': 0.09117613732814789,
})

# FR 168
informer_results.append({
    'Country': 'FR',
    'Pred_len': 168,
    'Iteration': 1,
    'MSE': 0.026063205674290657,
    'RMSE': 0.16144102811813354,
    'MAE': 0.10040990263223648,
})
informer_results.append({
    'Country': 'FR',
    'Pred_len': 168,
    'Iteration': 2,
    'MSE': 0.026005728170275688,
    'RMSE': 0.16126291453838348,
    'MAE': 0.10041678696870804,
})
informer_df = convert_results_into_df(informer_results, if_loss_fnc=False)

# Final DF
informer_df.columns = pd.MultiIndex.from_product([['Informer'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])

In [ ]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_transformers", 'informer_npy_168')
os.rename("test_results", "informer_pics_168")
path = 'results/informer'

if not os.path.exists(path):
    os.makedirs(path)

informer_df = convert_results_into_df(informer_results, if_loss_fnc=False)

# Final DF
informer_df.columns = pd.MultiIndex.from_product([['Informer'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
informer_df.to_csv(os.path.join(path, 'informer_168.csv'))
informer_df.round(4)

Model            Informer                
Metrics               MSE    RMSE     MAE
Country Pred_len                         
DE      24         0.0257  0.1603  0.1028
        96         0.0427  0.2066  0.1433
        168        0.0503  0.2243  0.1545
ES      24         0.0267  0.1632  0.0977
        96         0.0608  0.2465  0.1540
        168        0.0550  0.2343  0.1495
FR      24         0.0130  0.1139  0.0647
        96         0.0222  0.1491  0.0907
        168        0.0239  0.1547  0.0971
GB      24         0.0366  0.1903  0.1242
        96         0.0557  0.2361  0.1630
        168        0.0570  0.2386  0.1661
IT      24         0.0122  0.1102  0.0646
        96         0.0211  0.1451  0.0898
        168        0.0220  0.1482  0.0937

# 3. PatchTST 168

We separated PatchTST from Informer, because it has additional arguments. It is not so easy to modify f-string (as e. g. distionary) to unpack some arguments with if statement. Moreover, it has different parameter values.

In [ ]:
log_dir = f"logs/patchtst/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
seq_len = 168
model = "PatchTST"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_168.log"

# Parameters for tuning,but default
lr = 0.0001
n_heads = 16
e_layers = 3
d_model = 128
d_ff = 256
dropout = 0.2
batch_size = 128 

# List to store the results
patchtst_results = []

In [ ]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=128, patience=10, learning_rate=0

In [ ]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_transformers", 'patchtst_npy_168')
os.rename("test_results", "patchtst_pics_168")

path = 'results/patchtst'

if not os.path.exists(path):
    os.makedirs(path)

patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['PatchTST/21'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_168_21_patch.csv'))
patchtst_df.round(4)

Model            PatchTST/42                
Metrics                  MSE    RMSE     MAE
Country Pred_len                            
DE      24            0.0210  0.1448  0.0877
        96            0.0377  0.1941  0.1283
        168           0.0393  0.1981  0.1339
ES      24            0.0098  0.0988  0.0592
        96            0.0185  0.1361  0.0861
        168           0.0207  0.1439  0.0916
FR      24            0.0100  0.1000  0.0548
        96            0.0193  0.1390  0.0808
        168           0.0213  0.1460  0.0868
GB      24            0.0250  0.1582  0.0996
        96            0.0418  0.2045  0.1395
        168           0.0447  0.2115  0.1468
IT      24            0.0101  0.1004  0.0567
        96            0.0188  0.1371  0.0807
        168           0.0210  0.1448  0.0872